## [1] 논문 요약 /w BART

In [1]:
!pip install transformers
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.8 MB/s eta 0:00:00


PDF 텍스트 추출 + chunk 나누기

In [2]:
import fitz  # PyMuPDF
from transformers import pipeline

# 논문 텍스트 추출
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text() # 각 페이지에서 텍스트 추출
    return full_text # 모든 페이지를 이어붙여 전체 논문 텍스트 획득

# 텍스트를 BART의 최대 입력 길이인 1024 tokens 이하 chunck로 나누는 함수 (긴 논문은 쪼개서 처리해야 함)
def split_text(text, max_length=1000):
    import textwrap
    return textwrap.wrap(text, width=max_length)

distilBART 모델로 요약

In [3]:
# 요약 파이프라인
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize_chunks(chunks):
    summaries = []
    for i, chunk in enumerate(chunks):
        # 생성될 요약 문장의 최대 길이: 200, 최소 요약 길이: 50, deterministic하게 생성하여 일관된 요약 생성
        summary = summarizer(chunk, max_length=200, min_length=50, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries # 요약 결과는 chunck 수만큼 나옴

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Project Report Predicting Used Car Price.pdf"  # 파일 업로드 후 경로 설정
text = extract_text_from_pdf(pdf_path)
chunks = split_text(text, max_length=1000)
summaries = summarize_chunks(chunks)

for idx, summary in enumerate(summaries):
    print(f"\n[요약 {idx+1}]\n{summary}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 200, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)



[요약 1]
 CS 229 Project Report: Predicting Used Car Prices . Kshitij Kumbar, Pranav Gadre and Varun Nayak develop machine learning models that can accurately predict the price of a used car based on its features . We implement and evaluate various learning methods on a dataset of the sale prices of    different makes and models across cities in the United States .

[요약 2]
 Deciding whether a used car is worth the posted price when you see listings online can be difficult . Several factors, including mileage, make, model, year, etc. can influence the actual worth of a car . The aim is to use machine learning algorithms to develop models for predicting used car prices .

[요약 3]
 We noticed that the dataset had many outliers due to large price sensitivity of used cars . This is because accident history and condition can have a significant effect on the car’s price . We converted the Make, Model and State into one-hot vectors to remove outliers .

[요약 4]
 CS 229 Project Report by Kshitij K

## [2] 기능 개요: 유사 논문 검색

In [ ]:
!pip install transformers faiss-cpu
!pip install transformers faiss-cpu requests tqdm

1. 논문 데이터 준비

In [ ]:
import requests
import xml.etree.ElementTree as ET

# arXiv에서 CS 분야 최신 논문 100개 불러오기
def fetch_arxiv_papers(query="cs", max_results=100):
    url = f"http://export.arxiv.org/api/query?search_query=cat:{query}&start=0&max_results={max_results}"
    response = requests.get(url)
    root = ET.fromstring(response.content)
    entries = root.findall("{http://www.w3.org/2005/Atom}entry")

    papers = []
    for entry in entries:
        title = entry.find("{http://www.w3.org/2005/Atom}title").text.strip().replace('\n', ' ')
        abstract = entry.find("{http://www.w3.org/2005/Atom}summary").text.strip().replace('\n', ' ')
        papers.append({"title": title, "abstract": abstract})
    return papers


2. E5 임베딩 모델 로드

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "intfloat/e5-small-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# 텍스트를 벡터화하는 함수
def embed(texts):
    # E5 모델은 입력 앞에 문맥 prefix 필요
    inputs = tokenizer([f"passage: {t}" for t in texts], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0]  # [CLS] 토큰 벡터
        return embeddings.cpu()


3. 논문 데이터 벡터화 & FAISS로 검색 인덱스 만들기

In [ ]:
import faiss
from tqdm import tqdm

# 논문 불러오기
docs = fetch_arxiv_papers(query="cs.LG", max_results=100)
doc_texts = [doc["title"] + " " + doc["abstract"] for doc in docs]

# 임베딩 벡터 생성
doc_embeddings = embed(doc_texts)

# FAISS 인덱스 구성
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension) # L2 거리 기반 검색
index.add(doc_embeddings.numpy())


4. 사용자 입력 -> 유사 논문 추천

In [ ]:
def search_similar_docs(query_text, top_k=3):
    query_embedding = embed([f"query: {query_text}"])
    D, I = index.search(query_embedding.numpy(), top_k)  # 거리 D, 인덱스 I
    results = []
    for i in I[0]:
        results.append(docs[i])
    return results

In [ ]:
query = "How to predict used car prices using machine learning?" # 자연어 질문도 가능
results = search_similar_arxiv(query)

for i, r in enumerate(results, 1):
    print(f"\n🔎 유사 논문 {i}")
    print(f"📄 제목: {r['title']}")
    print(f"📝 요약: {r['abstract']}")